<a href="https://colab.research.google.com/github/NicoPasche/WebScrappingUndefined_ba-/blob/main/botUndf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kora

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from kora.selenium import wd
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
DATA_TALLES = ['XS','S','M','L','XL','XXL']
DATA_FUNDAS_COLOR = ['ROSA','NARANJA','VERDE','VIOLETA','ROJO']
DATA_FUNDAS_TAMANO = ['IPHONE 11','IPHONE 11 PRO','IPHONE 11 PRO MAX','IPHONE 12','IPHONE 12 PRO/IPHONE 12 PRO MAX','IPHONE 13 PRO','IPHONE 13 PRO MAX']
DATA_FUNDAS_TAMANO_2 = ['IPHONE 11','IPHONE 11 PRO','IPHONE 11 PRO MAX','IPHONE 12','IPHONE 12 PRO/IPHONE 12 PRO MAX','IPHONE 13','IPHONE 13 PRO','IPHONE 13 PRO MAX']
def extractor_url(itemListaURL):
  for itemURL in itemListaURL:
      if itemURL[0:4] == "http":
        return itemURL
def extractor_stock(stocks):
  for itemStock in stocks:
    if itemStock == "No":
      return 0
    elif len(itemStock)<=4:
      return int(itemStock)

def extractro_stock_individual(stockDeTodoSF):
  contador = 0
  row = []
  rows = []
  tabla = []
  for auxiliar in stockDeTodoSF[1:]:
    row.append(auxiliar_extracto_stock_individual(auxiliar))
    contador = contador + 1
  rows.append(row)
  if contador == 7:
    tabla = DATA_FUNDAS_TAMANO
  elif contador > 8:
    for i in DATA_FUNDAS_TAMANO_2:
      for j in DATA_FUNDAS_COLOR:
        k = i + " " +j 
        tabla.append(k)
  elif contador == 1:
    tabla.append("TALLE ÚNICO/ITEM ÚNICO")       
  else:
    tabla = DATA_TALLES[:contador]
  df = pd.DataFrame.from_records(rows,columns=tabla)
  return df
      
def auxiliar_extracto_stock_individual(auxiliar):
  valor = '' 
  for numero in auxiliar:
    if numero == ",":
      return int(valor) 
    else:
      valor = valor + numero 

def extraer_stocks(URL):
  wd.get(URL)
  soup = BeautifulSoup(wd.page_source)
  stocks = str(soup.find("meta",{"name":"twitter:data2"})).split('"')
  stock = extractor_stock(stocks)
  stockDeTodo = (soup.findAll("script",{"type":"text/javascript"}))
  stockDeTodo = ''.join(map(str,stockDeTodo))
  stockDeTodoSF = (str(stockDeTodo).split('"stock":'))
  stockIndividual = extractro_stock_individual(stockDeTodoSF)
  return [stock,stockIndividual]
def crear_tablas(URL):
  wd.get(URL)
  wd.set_window_size(1920,10080)
  soup = BeautifulSoup(wd.page_source)
  items = soup.find("div",{"class":"row-fluid container-wide container-xs container-product-grid"})
  rows_processed = []
  for item in items.findAll("div",{"class":"span3 item-container m-bottom-half"}):
    itemTitle = item.find('a',{"class":"js-item-name item-name"})
    itemTitle = itemTitle.text.strip('\n                                                    ')
    itemListaURL = str(item.find('a',{"title":itemTitle})).split('"')
    itemURL = extractor_url(itemListaURL)
    boolHayStock = bool(item.find('div',{"style":"display:none;"}))
    row = []
    row.append(itemTitle)
    if (boolHayStock):
      row.append("Hay Stock")
      stock = extraer_stocks(itemURL)
      row.append(stock[0])
      row.append(stock[1].to_dict('records'))
    else:
      row.append("Sin Stock")
      stock = 0
      row.append(stock)
      row.append(stock)
    rows_processed.append(row)  
  df = pd.DataFrame.from_records(rows_processed,columns=["Item","Hay Stock?","Total de Stock","Individuales"])
  
  return df

In [3]:
 def main():
  
  URL = "https://undefinedba.com/productos/?mpage=6"
  tabla_uno = crear_tablas(URL)
  #tabla_uno.to_csv('raw_data.csv', index=False, header=True)
  while True:
    tabla_dos = crear_tablas(URL)
    auxiliar = tabla_dos
    #tabla_dos.to_csv('raw_data2.csv', index=False, header=True)
    if len(tabla_uno)>len(tabla_dos):
      print("Sacaron un item")
    elif len(tabla_uno)==len(tabla_dos):
      tabla_dos['Cambios de Stock'] = np.where( (tabla_dos['Total de Stock'] == tabla_dos['Item']) == ( tabla_uno['Total de Stock'] == tabla_dos['Item']), 'False', 'True')
      stock_anterior = tabla_uno['Total de Stock'] 
      stock_anterior = pd.Series(stock_anterior) 
      tabla_dos = tabla_dos.assign(stock_anterior=stock_anterior.values) 
      print(tabla_dos.to_string(index=False))
      auxiliar.pop("Cambios de Stock")
      time.sleep(15)
    else:
      print("Agregaron un item") 
    del tabla_uno
    tabla_uno = auxiliar
    del auxiliar
    del tabla_dos
  return 0


In [ ]:
if __name__ == '__main__':
  main()